<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_Pacman_ATARI_labels_020420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Installation and Imports

In [1]:
!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/mila-iqia/atari-representation-learning.git to /tmp/pip-req-build-pp9hlf2v
  Running command git clone -q git://github.com/mila-iqia/atari-representation-learning.git /tmp/pip-req-build-pp9hlf2v
  Created wheel for atariari: filename=atariari-0.0.1-cp36-none-any.whl size=46584 sha256=42e32ce8cf883dccc44930dc9016c18ffefcc5a58c8f9e98e8c00505bdecf42e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ftfbevo/wheels/3d/69/51/5e436e5ae566c5b4dec5c53e65396d516459877a42a11d7aa4
Successfully built atariari
  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-tylcei0z
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-tylcei0z
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=0915c83e024d4603b78b28a6fdf1384a0e992a4b6d3939aad50b5f1badc0cd00
  Stored in directory: /tmp/pip-ephem-wheel-cache-8rh7y04s/wheels/91/52/02/ec5c530fd76d56a66

In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time

from atariari.benchmark.envs import *
from torch.autograd import Variable

import gym
from atariari.benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [4]:
!wandb login ###########

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [5]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")

W&B Run: https://app.wandb.ai/nerdk312/ATARI_LABELS_MPC/runs/1479cdki

# Hyperparameters

In [0]:
ENV_NAME = 'MsPacmanNoFrameskip-v4'

feature_size = 14
#feature_size = all_defaults['feature_size'] # Nawid- Dimensionality of the representation
# workers = 8 # Nawid - Choosing the number of workers for the network

# Main loop hyperp
AGGR_ITER = 10
STEPS_PER_AGGR = 500

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 1000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Supervised Model Hyperp
ENV_LEARNING_RATE = 1e-4
REW_LEARNING_RATE = 1e-4
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  55

# Controller Hyperp
HORIZON_LENGTH = 5
NUM_ACTIONS_SEQUENCES = 200

observation_channels = 1
action_dim = 1
n_actions = 5 #  9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}.{}.{}".format(now.day, now.hour, now.minute, now.second)

# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

# Model setup

In [0]:
class EarlyStopping_loss(object):
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=5, verbose=False, wandb=None, name=""):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = 1e11 # Nawid - Set a very high initial best loss
        self.name = name
        self.wandb = wandb

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score >= self.best_score: # Nawid - Inverse signs to take into minimising loss instead of maximising accuracy
            self.counter += 1
            print(f'EarlyStopping for {self.name} counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                print(f'{self.name} has stopped')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(
                f'Validation loss decreased/improved for {self.name}  ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')

        save_dir = self.wandb.run.dir
        torch.save(model.state_dict(), save_dir + "/" + self.name + ".pt")
        self.wandb.save(save_dir + "/" + self.name + ".pt")
        self.val_loss_min = val_loss


In [0]:
class NNDynamicsModel(nn.Module):
    '''
    Model that predict the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicsModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, obs_output_dim)
        )
    def forward(self, x):
        return self.mlp(x.float())

class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, reward_output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x.float())

def model_MSEloss(y_pred,y_truth, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

def model_CEloss(y_pred,y_truth,device):
    '''
    Compute the CEloss
    '''
    y_truth = torch.Tensor(np.array(y_truth)).to(device)
    return F.cross_entropy(y_pred, y_truth)

def model_BCEloss(y_pred,y_truth,device):
    '''
    Compute the BCE (Binary cross entropy)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.binary_cross_entropy(y_pred.view(-1).float(), y_truth.view(-1))

# General Functions

In [0]:
def one_hot(i):
    a = np.zeros(n_actions, 'uint8')
    a[i] = 1
    return a

In [0]:
def normalise(train_data, val_data, scaler = None): # Nawid - Used to normalise each dimension individually
    if scaler is None:
        scaler = StandardScaler()
        train_data = scaler.fit_transform(train_data)
    else: 
        train_data = scaler.transform(train_data)
        
    val_data = scaler.transform(val_data)
    return train_data, val_data, scaler

def normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data,X_env_obs_scaler = None,y_env_scaler=None,X_rew_obs_scaler= None):
    # Unpack data
    (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    (X_rew_train_obs, X_rew_train_act, y_rew_train), (X_rew_val_obs, X_rew_val_act, y_rew_val) = rew_train_data, rew_val_data
    
    # Normalise training and validation data
    X_env_train_obs,X_env_val_obs, X_env_obs_scaler =  normalise(X_env_train_obs, X_env_val_obs, X_env_obs_scaler)
    y_env_train, y_env_val, y_env_scaler = normalise(y_env_train, y_env_val, y_env_scaler)
    X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler = normalise(X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler)
    
    # Concatentates the normalised states with the one hot vector for the actions
    X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
    X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)
    X_rew_train = np.concatenate((X_rew_train_obs,X_rew_train_act),axis=1)
    X_rew_val = np.concatenate((X_rew_val_obs,X_rew_val_act),axis=1)

    # Pack data tuples
    env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
    rew_train_data, rew_val_data = (X_rew_train, y_rew_train),(X_rew_val, y_rew_val)

    return env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_obs_scaler, y_env_scaler, X_rew_obs_scaler

#normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)

# Data collection and preprocessing

In [11]:
class Data_collection():
    def __init__(self,ENV_NAME,feature_size,state_mode = 'ATARIARI', encoder = None):
        self.ENV_NAME = ENV_NAME
        self.state_mode = state_mode
        self.feature_size = feature_size
        self.encoder = encoder
        self.repeated_initial = True # Nawid - set repeated initial as true initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def gather_random_trajectories(self,num_traj):

        dataset_random = []
        #Env name could either be the RAM case or the generic case
        env = AtariARIWrapper(gym.make(self.ENV_NAME)) 
        i = 0
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number :',n)
            # Initial set up
            obs = env.reset()
            self.repeated_initial = True # Nawid- Used to represent the initial state
            initial_info_labels = env.labels()
            info_labels = env.labels() # Nawid -  Used to get the current state
            #print('trajectory number',n)
            
            while True:
                # Choosing action and env step
                sampled_action = np.random.randint(0,n_actions)
                sampled_action_one_hot = one_hot(sampled_action)
                next_obs, reward, done, next_info = env.step(sampled_action)
                #print('done after env step', done)
                next_info_labels = next_info['labels']
                # self.repeated initial is set to true at first and it is is turned to true in the reward_collection class when done is true

                if self.repeated_initial: # If the initial state is repeating
                    if initial_info_labels == next_info_labels: # Current state is still the same as the initial state
                        pass
                    else:
                        # New state achieved, so save data
                        self.repeated_initial = False
                    
                        state, next_state = self.state_collection(obs, next_obs,info_labels, next_info_labels)
                        reward, pacman_done = self.reward_collection(reward, done, info_labels, next_info_labels)
                        #print('pacman_done', pacman_done)
                        dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                    
                        obs = next_obs
                        info_labels = next_info_labels

                else:
                    # Save data as this is when the initial state and the next state are not identical
                    state, next_state = self.state_collection(obs, next_obs,info_labels, next_info_labels)
                    reward, pacman_done = self.reward_collection(reward, done, info_labels, next_info_labels) # Nawid - Changed name to pacman done as the variable done is used to exit the loop
                    dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                
                    obs = next_obs
                    info_labels = next_info_labels
                    # Used to test whether the done behaviour was correct
                    
                    '''if pacman_done:
                        i +=1 
                        print('pacman done', i)'''
                    

                if done:
                    break 
        '''               
        #dataset_random = np.array(dataset_random)
        first_dataset = np.array(dataset_random[0])
        print(first_dataset[0])
        print('zero',dataset_random[0])
        print('One',dataset_random[1])
        print('Two',dataset_random[2])
        print('Three',dataset_random[3])
        print('Four',dataset_random[4])
        #print(dataset_random.shape)
        '''
        return dataset_random
    

    def state_collection(self,obs, next_obs, info_labels, next_info_labels):
        if self.state_mode == 'ATARIARI':
            state = []
            next_state = []
            i = 0 
            for key in info_labels :
                if i < self.feature_size: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    next_state.append(next_info_labels[key])
                    i +=1
                else:
                    state = np.array(state)
                    next_state = np.array(next_state)
                    return state, next_state
    
        elif self.state_mode == 'STDIM': # Encoding is the state
            assert self.encoder is not None
            with torch.no_grad():
                self.encoder.eval()
                state = self.encoder(obs.float().to(self.device) / 255)
                next_state = self.encoder(next_obs.float().to(self.device) / 255)
                return state.cpu().detach().numpy(), next_state.cpu().detach().numpy()

        else: # Image observations are the state or the RAM labels
            return obs, next_obs

    def reward_collection(self,reward,done,info_labels,next_info_labels):
        # checks whether there has been a change in lives- change done to 1, otherwise the done should be fine regardless
        if not info_labels['num_lives'] == next_info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
            #print('change in lives')
            done = True 
            self.repeated_initial = True
        return reward, done

    def unison_shuffled_copies(self,*args): # Nawid- Randomises all the different values, using *args to use a variable number of parameters so i can shuffle many different values at once
        p = np.random.permutation(len(args[0]))
        shuffled = [i[p] for i in args]
        return shuffled
    
    def collate_data(self,random_dataset, rl_dataset):
        if len(rl_dataset) > 0:
                
            random_dataset = np.array(random_dataset)
            rl_dataset = np.array(rl_dataset)
            print('random_dataset', random_dataset.shape)
            print('rl dataset', rl_dataset.shape)
        
            d_concat = np.concatenate([random_dataset, rl_dataset], axis=0)
            print('d_concat',d_concat.shape)        
        else:
            d_concat = np.array(random_dataset)
        
        num_examples_added = len(d_concat)

        # Split the dataset into train(80%) and test(20%)
        D_train = d_concat[:int(-num_examples_added*1/5)]
        D_valid = d_concat[int(-num_examples_added*1/5):]

        print("len(D):", len(d_concat), 'len(Dtrain)', len(D_train))

        # Shuffle the dataset
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_act = np.array([act for _,_,_,_,act in D_train])

        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        y_env_train = y_env_train - np.array([obs for obs,_,_,_,_ in D_train]) # y(state) = s(t+1) - s(t)

        # Reward's output
        y_rew_train = np.array([[D] for _,_,_,D,_ in D_train])
    
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val - np.array([obs for obs,_,_,_,_ in D_valid]) # y(state) = s(t+1) - s(t)

        # Reward output
        y_rew_val = np.array([[D] for _,_,_,D,_ in D_valid])
    
        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        rew_train_data, rew_val_data = (X_train_obs, X_train_act, y_rew_train), (X_val_obs, X_val_act, y_rew_val)

        return env_train_data, env_val_data, rew_train_data, rew_val_data 
    



data_collector = Data_collection('MsPacmanNoFrameskip-v4',feature_size)
random_data = data_collector.gather_random_trajectories(5)


trajectory number : 0


# Train model

In [0]:
def train_model(train_data,val_data,model,batch_size, max_model_iter, optimizer, device,early_stopper,desired_model='Env_model'):
    ''' 
    General function to train either of the two models
    '''
    # Unpack data
    (X_train, y_train), (X_val, y_val) =  train_data, val_data
    losses_env = []

    # Choose loss function based on what type of model is training
    if desired_model =='Env_model': # Nawid -  Decides which loss function to use
        loss_function = model_MSEloss
    else:
        loss_function = model_BCEloss
        #print('BCE being used')

    # go through max_model iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0,len(X_train), batch_size): # Nawid- Batch size is the step size
            if len(X_train) > mb + BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                y_mb = y_train[mb:mb+BATCH_SIZE]
                #X_mb += np.random.normal(loc = 0, scale = 0.001, size= X_mb.shape)

                optimizer.zero_grad()
                # forward pass of model to compute the output
                pred_mb = model(torch.tensor(X_mb).to(device))
                
                # compute the loss
                loss = loss_function(pred_mb,y_mb,device) #Nawid-  Uses Mse loss if dynamics model or uses BCE loss if reward/done model
                wandb.log({'{} Training loss'.format(desired_model):loss.cpu().detach().numpy()})
                # backward pass
                loss.backward()
                # optimization step
                optimizer.step()
        
        # Nawid - Calculate the validation loss after each epoch
        model.eval()
        with torch.no_grad():
            pred_val = model(torch.tensor(X_val).to(device))
            val_loss = loss_function(pred_val, y_val,device)
            wandb.log({'{} Validation loss'.format(desired_model):val_loss})
        

        # Checks whether to early stop after each epoch
        early_stopper(val_loss,model)
        if early_stopper.early_stop:
            print("Early stopping")
            break

# Controller

In [0]:
class multi_model_based_control():
    def __init__(self,ENV_NAME, env_model, rew_model,num_sequences, horizon_length, n_actions, norm, num_features = feature_size, state_mode ='ATARIARI'):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.state_mode = state_mode
        if self.state_mode == 'ATARIARI':
            self.env = AtariARIWrapper(gym.make(ENV_NAME))
            self.initial_info_labels = self.env.labels()
        
        self.env_model = env_model
        self.rew_model = rew_model
        self.horizon_length = horizon_length
        self.num_sequences = num_sequences
        self.repeated_initial = True # Nawid - set repeated initial as true initially
        self.env_input_scaler, self.env_output_scaler, self.rew_input_scaler = norm
        self.n_actions = n_actions
        self.num_features = num_features
        self.state_mode = state_mode
        


    def random_sampling_shooting(self,real_obs):
        '''
        Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
        '''
        best_reward = -1e9
        best_next_action = []
        m_obs = np.array([real_obs for _ in range(self.num_sequences)])

        # array that contains the rewards for all the sequence
        unroll_rewards = np.zeros((self.num_sequences, 1)) 
        first_sampled_actions = []

        self.env_model.eval()
        self.rew_model.eval()

        # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
        ## i.e roll a given number of trajectories in a single batch (to increase speed)
        for t in range(self.horizon_length):
            # sample actions for each sequence
            sampled_actions = np.array([np.random.randint(0,self.n_actions) for _ in range(self.num_sequences)])
            sampled_actions_one_hot = np.array([one_hot(action) for action in sampled_actions])
            if isinstance(self.env_model,NNDynamicsModel): # Nawid-  If the env model is a neural net
                #print('using dynamics model')
                m_obs_env_scaled = self.env_input_scaler.transform(m_obs)
                env_model_input = np.concatenate([m_obs_env_scaled, sampled_actions_one_hot], axis = 1)
                # compute the next state for each sequence
                pred_obs = self.env_model(torch.tensor(env_model_input).to(self.device))

                # inverse scaler transformation
                pred_obs = self.env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
                # add previous observation
                next_obs = pred_obs + m_obs
            else:
                #print('using oracle state')            
                next_obs = self.env_model.predict_states(m_obs,sampled_actions) # Nawid - Able to obtain the next state directly rather than predicting a change in states

            if isinstance(self.rew_model, NNRewardModel):
                m_obs_rew_scaled = self.rew_input_scaler.transform(m_obs)
                rew_model_input = np.concatenate([m_obs_rew_scaled, sampled_actions_one_hot], axis = 1)
                pred_rew = self.rew_model(torch.tensor(rew_model_input).to(self.device)) # Nawid -  I believe I do not need to rescale for a True of false situation
                unroll_rewards += (1 - pred_rew.cpu().detach().numpy())
            else:
                #print('using oracle reward')
                pred_rew = self.rew_model.predict_reward(m_obs, sampled_actions,next_obs)
                unroll_rewards += pred_rew
        
            m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards

            if t ==0:
                first_sampled_actions = sampled_actions
        
        self.env_model.train()
        self.rew_model.train()

        # Best the position of the sequence with the higher reward
        arg_best_reward = np.argmax(unroll_rewards)
        best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
        # take the first action of this sequence
        best_action = first_sampled_actions[arg_best_reward]
        #best_action =  np.squeeze(best_action)
        return best_action, best_sum_reward
    
    def check_initial(self, next_info_labels):
        # Checks if there has been a change from the initial info labels to show that the initial lag period is over 
        if self.initial_info_labels == next_info_labels:
            pass
        else:
            self.repeated_initial = False
    
    def state_conversion(self,obs, info_labels):
        if self.state_mode == 'ATARIARI':
            state = []
            i = 0 
            for key in info_labels :
                if i < self.num_features: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    i +=1
                else:
                    state = np.array(state)
                    return state
    
        elif self.state_mode == 'STDIM': # Encoding is the state
            assert self.encoder is not None
            with torch.no_grad():
                self.encoder.eval()
                state = self.encoder(obs.float().to(self.device) / 255)
                return state.cpu().detach().numpy()
        else: # Image observations are the state or the RAM labels
            return obs
    
    def reward_conversion(self,reward,done,info_labels,next_info_labels):
        # checks whether there has been a change in lives- change done to 1, otherwise the done should be fine regardless
        if not info_labels['num_lives'] == next_info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
            done = True 
            self.repeated_initial = True
            self.initial_info_labels  = next_info_labels # Sets the new initial labels.
        return reward, done

# Saving config

In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.train_model_iter = TRAIN_ITER_MODEL 
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR
config.env_lr = ENV_LEARNING_RATE
config.rew_lr = REW_LEARNING_RATE
config.no_actions = n_actions

# Main - Model training

Data collection and processing

In [32]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,feature_size)
#rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
rand_dataset = data_collector.gather_random_trajectories(100)
rl_dataset = []
env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)

norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
norm = (X_env_state_scaler, y_env_scaler, X_rew_state_scaler)


trajectory number : 0
trajectory number : 10
trajectory number : 20
trajectory number : 30
trajectory number : 40
trajectory number : 50
trajectory number : 60
trajectory number : 70
trajectory number : 80
trajectory number : 90
len(D): 168620 len(Dtrain) 134896


Dynamics model training

In [30]:
env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(env_model, log="all")
env_model_name = 'Env_model'+ '_' + date_time
early_stopping_env = EarlyStopping_loss(patience=7, verbose=True, wandb=wandb, name=env_model_name)

for n_iter in range(AGGR_ITER):
    if early_stopping_env.early_stop:
        print('Early stopping')
        break
    train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

  5%|▌         | 3/55 [00:00<00:27,  1.91it/s]

Validation loss decreased/improved for Env_model_5_17.14.56  (100000000000.000000 --> 0.674989).  Saving model ...
Validation loss decreased/improved for Env_model_5_17.14.56  (0.674989 --> 0.672204).  Saving model ...
Validation loss decreased/improved for Env_model_5_17.14.56  (0.672204 --> 0.669794).  Saving model ...


  9%|▉         | 5/55 [00:00<00:19,  2.58it/s]

Validation loss decreased/improved for Env_model_5_17.14.56  (0.669794 --> 0.667757).  Saving model ...
Validation loss decreased/improved for Env_model_5_17.14.56  (0.667757 --> 0.666063).  Saving model ...
Validation loss decreased/improved for Env_model_5_17.14.56  (0.666063 --> 0.664676).  Saving model ...


 16%|█▋        | 9/55 [00:01<00:10,  4.42it/s]

...

EarlyStopping for Env_model_5_17.14.56 counter: 1 out of 7
EarlyStopping for Env_model_5_17.14.56 counter: 2 out of 7
EarlyStopping for Env_model_5_17.14.56 counter: 3 out of 7


 69%|██████▉   | 38/55 [00:02<00:01, 13.75it/s]

EarlyStopping for Env_model_5_17.14.56 counter: 4 out of 7
EarlyStopping for Env_model_5_17.14.56 counter: 5 out of 7
EarlyStopping for Env_model_5_17.14.56 counter: 6 out of 7


 69%|██████▉   | 38/55 [00:02<00:01, 12.93it/s]


EarlyStopping for Env_model_5_17.14.56 counter: 7 out of 7
Env_model_5_17.14.56 has stopped
Early stopping
Early stopping


Reward model training

In [0]:
rew_model = NNRewardModel(n_actions + feature_size, reward_dim).to(device) # Nawid - Need to put both actions as it is a one-hot vector
rew_optimizer = torch.optim.Adam(rew_model.parameters(),REW_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(rew_model, log="all")
Rew_model_name = 'Rew_model'+ '_' + date_time
early_stopping_rew = EarlyStopping_loss(patience=7, verbose=True, wandb=wandb, name=Rew_model_name)

for n_iter in range(AGGR_ITER):
    if early_stopping_rew.early_stop:
        print('Early stopping')
        break
    train_model(norm_rew_train_data, norm_rew_val_data, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_rew,desired_model='Rew_model')


    

  2%|▏         | 1/55 [00:16<14:33, 16.17s/it]

Validation loss decreased/improved for Rew_model_4_19.8.50  (100000000000.000000 --> 0.020023).  Saving model ...


  4%|▎         | 2/55 [00:31<14:02, 15.89s/it]

Validation loss decreased/improved for Rew_model_4_19.8.50  (0.020023 --> 0.011873).  Saving model ...


  5%|▌         | 3/55 [00:46<13:32, 15.62s/it]

...

EarlyStopping for Rew_model_4_19.8.50 counter: 4 out of 7


 40%|████      | 22/55 [05:30<08:07, 14.77s/it]

EarlyStopping for Rew_model_4_19.8.50 counter: 5 out of 7


 42%|████▏     | 23/55 [05:44<07:52, 14.75s/it]

EarlyStopping for Rew_model_4_19.8.50 counter: 6 out of 7
EarlyStopping for Rew_model_4_19.8.50 counter: 7 out of 7
Rew_model_4_19.8.50 has stopped
Early stopping
Early stopping


# Training the reward function and MPC

In [31]:
rew_model = NNRewardModel(n_actions + feature_size, reward_dim).to(device) # Nawid - Need to put both actions as it is a one-hot vector
rew_optimizer = torch.optim.Adam(rew_model.parameters(),REW_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(rew_model, log="all")
Rew_model_name = 'Rew_model'+ '_' + date_time
early_stopping_rew = EarlyStopping_loss(patience=7, verbose=True, wandb=wandb, name=Rew_model_name)

#env = AtariARIWrapper(gym.make(ENV_NAME))  # Instantiate environment
Controller = multi_model_based_control(ENV_NAME,env_model, rew_model, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH, n_actions, norm)

for n_iter in range(AGGR_ITER):
    if early_stopping_rew.early_stop:
        print('Early stopping')
        break    
    train_model(norm_rew_train_data, norm_rew_val_data, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_rew,desired_model='Rew_model')

    obs = Controller.env.reset()
    #obs = env.reset()
    #initial_info_labels = env.labels()

    num_examples_added = 0
    game_reward = 0
    controller_pred_rews = []
    rews = []
    i = 0 
    while num_examples_added < STEPS_PER_AGGR:
        while True:
            tt = time.time()
            
            if Controller.repeated_initial:
                i += 1
                print(i)
                obs,_,_,info = Controller.env.step(0) # Any action taken
                info_labels = info['labels']
                # Checks if the initial set is being repeated and if it isnt, it sets the value off
                Controller.check_initial(info_labels) 
            else:
                # new state achieved
                state = Controller.state_conversion(obs,info_labels)
                action, pred_rew = Controller.random_sampling_shooting(state)
                action_one_hot = one_hot(action)
                controller_pred_rews.append(pred_rew)

                # one step in the environment with the action returned by
                next_obs, reward,done, next_info = Controller.env.step(action)
                next_info_labels = next_info['labels']
                
                next_state = Controller.state_conversion(next_obs, next_info_labels)
                # Obtains the reward, done and sets whether repeated initial should be true or not
                reward, pacman_done = Controller.reward_conversion(reward,done,info_labels, next_info_labels) 

                # add to the RL dataset
                rl_dataset.append([state, next_state, reward, pacman_done, action_one_hot])
                num_examples_added += 1
                
                obs = next_obs
                info_labels = next_info_labels

                game_reward += reward 
                if done:
                    obs = Controller.env.reset()
                    print('  >> R: {:.2f}, Mean sum:{:.2f}, {}'.format(game_reward, np.mean(controller_pred_rews), num_examples_added))

                    rews.append(game_reward)
                    game_reward = 0
                    controller_pred_rews = []
                    break

    print('  >> Mean: {:.2f}', np.mean(rews))    

  5%|▌         | 3/55 [00:00<00:26,  1.97it/s]

Validation loss decreased/improved for Rew_model_5_17.14.56  (100000000000.000000 --> 0.740351).  Saving model ...
Validation loss decreased/improved for Rew_model_5_17.14.56  (0.740351 --> 0.706320).  Saving model ...
Validation loss decreased/improved for Rew_model_5_17.14.56  (0.706320 --> 0.664157).  Saving model ...


  9%|▉         | 5/55 [00:00<00:18,  2.65it/s]

Validation loss decreased/improved for Rew_model_5_17.14.56  (0.664157 --> 0.620620).  Saving model ...
Validation loss decreased/improved for Rew_model_5_17.14.56  (0.620620 --> 0.577853).  Saving model ...
Validation loss decreased/improved for Rew_model_5_17.14.56  (0.577853 --> 0.536675).  Saving model ...


 16%|█▋        | 9/55 [00:01<00:10,  4.45it/s]

...

  7%|▋         | 4/55 [00:00<00:04, 12.16it/s]

EarlyStopping for Rew_model_5_17.14.56 counter: 3 out of 7
EarlyStopping for Rew_model_5_17.14.56 counter: 4 out of 7
EarlyStopping for Rew_model_5_17.14.56 counter: 5 out of 7


 11%|█         | 6/55 [00:00<00:04, 10.95it/s]

EarlyStopping for Rew_model_5_17.14.56 counter: 6 out of 7
EarlyStopping for Rew_model_5_17.14.56 counter: 7 out of 7
Rew_model_5_17.14.56 has stopped
Early stopping
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170


171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420


# OLD CODE

In [0]:
# gather the dataset of random sequences

encoder = NatureCNN(observation_channels,all_defaults)
encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_55.pt'))
encoder.to(device)
encoder.eval()
rand_dataset = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
rl_dataset = []
num_examples_added = len(rand_dataset)
env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset,rl_dataset,num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
norm = (X_env_state_scaler, y_env_scaler, X_rew_state_scaler)

env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(env_model, log="all")
env_model_name = 'Env_model'+ '_' + date_time
early_stopping_env = EarlyStopping(patience=5, verbose=True, wandb=wandb, name=env_model_name)

for n_iter in range(AGGR_ITER):
    if early_stopping_env.early_stop:
        print('Early stopping')
        break
    train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

  4%|▎         | 2/55 [00:00<00:18,  2.91it/s]

Validation accuracy increased for Env_model_27_13.53.44  (0.000000 --> 1.118334).  Saving model ...
EarlyStopping for Env_model_27_13.53.44 counter: 1 out of 5


  7%|▋         | 4/55 [00:00<00:11,  4.37it/s]

EarlyStopping for Env_model_27_13.53.44 counter: 2 out of 5
EarlyStopping for Env_model_27_13.53.44 counter: 3 out of 5


  9%|▉         | 5/55 [00:01<00:10,  4.69it/s]

EarlyStopping for Env_model_27_13.53.44 counter: 4 out of 5
EarlyStopping for Env_model_27_13.53.44 counter: 5 out of 5
Env_model_27_13.53.44 has stopped
Early stopping
Early stopping


In [0]:
def gather_random_trajectories(num_traj,ENV_NAME,mode = 'ATARIARI', encoder = None):
    '''
    states_list = []
    next_states_list = []
    rewards_list = []
    dones_list = []
    actions_list = []
    '''
    dataset_random = []
    #Env name could either be the RAM case or the generic case
    env = AtariARIWrapper(gym.make(ENV_NAME)) 

    for n in range(num_traj):
        obs = env.reset()
        repeated_initial = True # Nawid- Used to represent the initial state
        initial_info_labels = env.labels()
        info_labels = env.labels() # Nawid -  Used to get the current state
        while True:
            sampled_action = np.random.randint(0,n_actions)
            sampled_action_one_hot = one_hot(sampled_action)
            next_obs, reward, done, next_info = env.step(sampled_action)
            next_info_labels = next_info['labels']

            if repeated_initial: # If the initial state is repeating
                if initial_info_labels == next_info_labels: # Current state is still the same as the initial state
                    pass
                else:
                    repeated_initial = False
                    
                    state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                    reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                    dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                    
                    obs = next_obs
                    info_labels = next_info_labels

                    # Go into a new state and from the previous state and the current state as this is a transition between the states

            else:
                # Save data as this is when the initial state and the next state are not identical
                state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                
                obs = next_obs
                info_labels = next_info_labels

                if not initial_info_labels['num_lives'] == info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
                    print('change in lives')
                    repeated_initial = True



In [0]:
def state_collection(obs, next_obs, info_labels, next_info_labels,mode='ATARIARI', encoder = None):
    if mode == 'ATARIARI':
        state = []
        next_state = []
        i = 0 
        for key in info_labels :
            if i < 14: # Nawid - Only the first 14 info are crucial I believe
                state.append(info_labels[key])
                next_state.append(info_labels[key])
                i +=1
            else:
                state = np.array(state)
                next_state = np.array(next_state)
                return state, next_state
    
    elif mode == 'STDIM': # Encoding is the state
        assert encoder is not None
        with torch.no_grad():
            encoder.eval()
            states = encoder(obs.float().to(device) / 255)
            next_states = encoder(next_obs.float().to(device) / 255)
            return states, next_states

    else: # Image observations are the state or the RAM labels
        return obs, next_obs

'''
env = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4')) 
info_labels = env.labels()
#print(info_labels)
state = []
print(info_labels)
for key in info_labels:
    state.append(info_labels[key])

state = np.array(state)
compact_state = state[0:14]
print(compact_state)
'''

In [0]:
# Instantiate reward model
env = gym.make('MsPacmanNoFrameskip-v4')
encoder = ImpalaCNN(observation_channels,all_defaults)
encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_95.pt'))
encoder.to(device)
encoder.eval()

rew_model = NNRewardModel(n_actions + feature_size,reward_dim).to(device)
rew_optimizer = torch.optim.Adam(rew_model.parameters(), REW_LEARNING_RATE)
wandb.watch(rew_model, log="all")
desired_model = 'Reward_model'
model_name = desired_model +'_'+ date_time
early_stopping_rew = EarlyStopping(patience=7, verbose=True, wandb=wandb, name=model_name)

game_reward = 0
rand_dataset = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
print('Does it work past here')
rl_dataset = []
num_examples_added = len(rand_dataset)

env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset, rl_dataset, num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
print('Is data normalised?')
#env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset,rl_dataset,num_examples_added)

for n_iter in range(AGGR_ITER):
    if early_stopping_reward.early_stop:
        print('Early stopping')
        break
    #norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler)
    print('Does the model train')
    train_model(norm_rew_train_data, norm_rew_val_data, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_rew)

  0%|          | 0/55 [00:00<?, ?it/s]

rewards list (10000, 1)
dones list (10000, 1)
Does it work past here
Is data normalised?
Does the model train


  5%|▌         | 3/55 [00:00<00:29,  1.78it/s]

Validation accuracy increased for Reward_model_2_9.26.46  (0.000000 --> 0.230905).  Saving model ...
EarlyStopping for Reward_model_2_9.26.46 counter: 1 out of 7
EarlyStopping for Reward_model_2_9.26.46 counter: 2 out of 7


  9%|▉         | 5/55 [00:01<00:21,  2.37it/s]

...

EarlyStopping for Reward_model_2_9.26.46 counter: 11 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 12 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train


  0%|          | 0/55 [00:00<?, ?it/s]

EarlyStopping for Reward_model_2_9.26.46 counter: 13 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 14 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train


  0%|          | 0/55 [00:00<?, ?it/s]

EarlyStopping for Reward_model_2_9.26.46 counter: 15 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 16 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping


In [0]:
# gather the dataset of random sequences

encoder = NatureCNN(observation_channels,all_defaults)
encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_55.pt'))
encoder.to(device)
encoder.eval()
rand_dataset = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
rl_dataset = []
num_examples_added = len(rand_dataset)
env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset,rl_dataset,num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
norm = (X_env_state_scaler, y_env_scaler, X_rew_state_scaler)

env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(env_model, log="all")
env_model_name = 'Env_model'+ '_' + date_time
early_stopping_env = EarlyStopping(patience=5, verbose=True, wandb=wandb, name=env_model_name)

for n_iter in range(AGGR_ITER):
    if early_stopping_env.early_stop:
        print('Early stopping')
        break
    train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)